In [1]:
# Import Libraries

import yfinance as yf
import pandas as pd
import numpy as np
import talib as ta

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# Tickers List
TICKERS = ["V", "INTC", "CAT", "LLY", "DIS", "TSLA", "GS", "BA", "CSCO","ABBV",
          "AMZN", "JPM", "KO", "MCD", "PYPL", "WMT", "IBM", "AMGN", "REGN", "JNJ"]
TICKER = 'MSFT'
START = '2021-01-01'

In [3]:
ticker = yf.Ticker(TICKER)
df = ticker.history(start=START)[['Open', 'Close', 'High', 'Low', 'Volume']]
df

,Open,Close,High,Low,Volume
Date,,,,,
2021-01-04 00:00:00-05:00,216.710025,211.996613,217.167734,209.191930,37130100
2021-01-05 00:00:00-05:00,211.577850,212.201111,212.804906,210.058652,23823000
2021-01-06 00:00:00-05:00,206.620989,206.698898,210.828012,206.397008,35930700
2021-01-07 00:00:00-05:00,208.442070,212.580917,213.603459,208.120714,27694500
2021-01-08 00:00:00-05:00,212.960681,213.876099,214.810998,211.353841,22956200
...,...,...,...,...,...
2023-12-18 00:00:00-05:00,369.450012,372.649994,373.000000,368.679993,21802900
2023-12-19 00:00:00-05:00,371.489990,373.260010,373.260010,369.839996,20603700
2023-12-20 00:00:00-05:00,375.000000,370.619995,376.029999,370.529999,26316700


In [4]:
TEMPLATE = 'plotly_dark'
def set_padding(fig):
    fig.update_layout(margin = go.layout.Margin(r=10,b=10))

def add_range_selector(fig):
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=[
                    dict(count=1, label='1m', step='month', stepmode='backward'),
                    dict(count=6, label='6m', step='month', stepmode='backward'),
                    dict(count=1, label='YTD', step='year', stepmode='todate'),
                    dict(count=1, label='1y', step='year', stepmode='backward'),
                    dict(step='all')
                ]),
            type='date'),
        xaxis2_type='date')


def add_volume_chart(fig):
    colors = ['#9C1F0B' if row['Open'] - row['Close'] >= 0
              else '#2B8308' for index, row in df.iterrows()]
    
    fig.add_trace(go.Bar(x=df.index, y=df['Volume'], showlegend = False, marker_color = colors), row = 2, col = 1)

In [5]:
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
dt_obs = [d.strftime("%y-%m-%d") for d in pd.to_datetime(df.index)]
dt_breaks = [d for d in dt_all.strftime("%y-%m-%d").tolist() if not d in dt_obs]

In [7]:
df_buy = df.query('Low < BL')[['Close']]
df_sell = df.query('High > BU')[['Close']]

df_buy['Close'] = df_buy['Close'].round(2)
df_sell['Close'] = df_sell['Close'].round(2)

In [9]:
analysis = pd.DataFrame()
analysis['macd'],analysis['macdSignal'], analysis['macdHist'] = ta.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)

fig8=make_subplots(rows=2,cols=1, vertical_spacing=0.25,shared_xaxes=True)
fig8.append_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'],
                                      close=df['Close'],showlegend=False, name='Candlestick'), row=1,col=1)

fig8.append_trace(go.Scatter(x=df.index, y=analysis['macd'], line=dict(color='#C42836', width=1),
                             name='MACD Line'), row=2,col=1)

fig8.append_trace(go.Scatter(x=df.index, y=analysis['macdSignal'], line=dict(color='limegreen', width=1),
                             name='Signal Line'), row=2,col=1)

colors = np.where(analysis['macd'] < 0, '#EA071C', '#57F219')

fig8.append_trace(go.Bar(x=df.index, y=analysis['macdHist'], marker_color=colors,
                             name='Histogram'), row=2,col=1)

# Update y  & x axis labels
fig8.update_yaxes(title_text='Price')
fig8.update_xaxes(title_text='Date')
    
# Change the Close to Price for the legend label
fig8.data[0].name = 'Price'

# Sets customized padding
set_padding(fig8)

fig8.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label='1m', step='month', stepmode='backward'),
            dict(count=6, label='6m', step='month', stepmode='backward'),
            dict(count=1, label='YTD', step='year', stepmode='todate'),
            dict(count=1, label='1y', step='year', stepmode='backward'),
            dict(step='all')
        ])
    )
)

# Make it pretty
layout8 = go.Layout(template=TEMPLATE, title=TICKER + ' - MACD Indicator', height=700, xaxis_rangeslider_visible = False,
                    xaxis1=dict(rangeselector=dict(font=dict(color='black'))),
                    xaxis2=dict(rangeselector=dict(font=dict(color='black'))))
fig8.update_layout(layout8)
fig8.update_layout(legend=dict(yanchor='top',y=0.45,xanchor='left',x=1.01))
fig8.show()